## Bedrock Knowledge Base - Retrieve API, Langchain, and LLaMa Index for Prompt Completion Evaluations - Utilizing LLaMa 70B Role Play Model + Titan for embeddings

In [1]:
#install knowledge base sdk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Processing ./SDK/boto3-1.28.84-py3-none-any.whl
  Using cached botocore-1.31.85-py3-none-any.whl.metadata (6.1 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.7.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using cached botocore-1.31.85-py3-none-any.whl (11.3 MB)
Using cached s3transfer-0.7.0-py3-none-any.whl (79 kB)
Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall:

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [2]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [8]:


import boto3
import pprint
from botocore.client import Config
from langchain.llms.bedrock import Bedrock
from llama_index import (
    ServiceContext,
    set_global_service_context
)
from langchain.embeddings.bedrock import BedrockEmbeddings
from llama_index.embeddings import LangchainEmbedding

pp = pprint.PrettyPrinter(indent=2)

ACCESS_KEY = 
SECRET_KEY = 

bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              region_name='us-east-1',
                              config=bedrock_config,
                              aws_access_key_id=ACCESS_KEY,
                              aws_secret_access_key=SECRET_KEY)

model_kwargs_meta = {
    "temperature": 0,
    "top_k": 10,
    "max_tokens_to_sample": 3000
}

embed_model = LangchainEmbedding(
    BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")
)

llm = Bedrock(model_id="meta.llama2-70b-chat-v1",
              model_kwargs=model_kwargs_meta,
              client = bedrock_client,)

service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model)
set_global_service_context(service_context)

### Retrieve API: Process flow 

Define a retrieve function that initializes the agent client to get a query that the user inputs, along with the knowledge base id they configured on the bedrock console, and the number of results they need to retrieve while performing RAG on the Knowledge base 

In [9]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults
            }
        }
    )

### Initialize your KB below with the KB_ID

In [10]:

kb_id =  

In [12]:
query = "What is Amazon EC2?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'Cloud computing provides a simple way to access '
                         'servers, storage, databases and a broad set of '
                         'application services over the Internet. A cloud '
                         'services platform such as Amazon Web Services owns '
                         'and maintains the network-connected hardware '
                         'required for these application services, while you '
                         'provision and use what you need via a web '
                         'application.   2        Overview of Amazon Web '
                         'Services AWS Whitepaper   Six Advantages of Cloud '
                         'Computing • Trade capital expense for variable '
                         'expense – Instead of having to invest heavily in '
                         'data centers   and servers before you know how '
                         'you’re going to use them, you can pay only when you '
            

In [14]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are an advanced AI system specialized in Amazon Web Services (AWS), capable of providing detailed and accurate information about various AWS services. 
Use the available resources and knowledge to answer the question enclosed in <question> tags. 
If the answer to a question is not within your current scope of knowledge, please indicate that you don't know, and do not attempt to speculate or fabricate a response.
<context>
{context_str}
</context>

<question>
{query_str}
</question>

Your response should be precise, detailed, and include any relevant AWS-specific terminology, features, or concepts. Utilize your extensive knowledge base about AWS services to provide the most accurate and current information available.

Assistant:"""

meta_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context_str","query_str"])

In [15]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in reversed(retrievalResults): 
        contexts.append(retrievedResult['content']['text'])
    return contexts

#### View all of the relevant chunks re ordered below from most to least relevant based on the embedding scores generated

In [16]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

[ 'You can easily sign up a 1- or 3-year term Savings Plans in AWS Cost '
  'Explorer and manage your plans by taking advantage of recommendations, '
  'performance reporting, and budget alerts.   Compute Services Topics   • '
  'Amazon EC2 (p. 20) • Amazon EC2 Auto Scaling (p. 21) • Amazon EC2 Image '
  'Builder (p. 21) • Amazon Lightsail (p. 22) • AWS App Runner (p. 22) • AWS '
  'Batch (p. 22) • AWS Elastic Beanstalk (p. 22) • AWS Fargate (p. 22) • AWS '
  'Lambda (p. 23) • AWS Serverless Application Repository (p. 23) • AWS '
  'Outposts (p. 23) • AWS Wavelength (p. 23) • VMware Cloud on AWS (p. 24)   '
  'Amazon EC2 Amazon Elastic Compute Cloud (Amazon EC2) is a web service that '
  'provides secure, resizable compute capacity in the cloud. It is designed to '
  'make web-scale computing easier for developers.   The simple web interface '
  'of Amazon EC2 allows you to obtain and configure capacity with minimal '
  'friction. It provides you with complete control of your computing